In [5]:
%matplotlib agg

import numpy as np
import matplotlib
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
from collections import deque


np.set_printoptions(precision=2) 
pd.set_option('display.max_rows', None)

DISTANCE = 45
N_devices = 20


def bfs(adj_matrix, start, end):
    
    num_nodes = len(adj_matrix)
    visited = [False] * num_nodes
    queue = deque()

    queue.append((start, [start]))  
    while queue:        
        current_node, path = queue.popleft()
        visited[current_node] = True
        if current_node == end:
            return path
        
        for neighbor in range(num_nodes):
            if adj_matrix[current_node][neighbor] and not visited[neighbor]:
                queue.append((neighbor, path + [neighbor]))

    return None


def distance(point0,point1):
    return np.sqrt((point0[0]-point1[0])**2+(point0[1]-point1[1])**2)

def get_adj(devices):
        num_devices = len(devices)
        adj_mat = np.zeros((num_devices,num_devices),dtype=np.bool8)
        for i,device in enumerate(devices):
            for j,device1 in enumerate(devices):
                if distance(device.get_cords(),device1.get_cords()) < DISTANCE and i !=j  :
                    adj_mat[i][j]=True
                    adj_mat[j][i]=True
        return adj_mat

        
class Message():
    def __init__(self,sender,reciver,message,t) -> None:
        self.message = message
        self.sender = sender
        self.reciver = reciver
        self.successd = "failed"
        self.time = t

    def tostring(self):
        return [f"message:{self.message},sender:{self.sender},reciver{self.reciver},succ:{self.successd}"]
        




class Device():
    def __init__(self,id,map,speed=1,color="ro") -> None:
        self.id = id
        self.battery = 100

        self.message_type = ["Hello","hi","hru","<3",":)"]

        self.pos_x = st.uniform(0,map[0]).rvs()
        self.pos_y = st.uniform(0,map[1]).rvs()

        self.memory = []

        self.mp2 = False

        self.dis_move = st.norm(0,speed)
        self.dis_send = st.bernoulli(0.1)
        self.dis_reciver = st.randint(0,N_devices-1)
        self.dis_type = st.randint(0,len(self.message_type))


        self.map = map
        self.color = color
        self.messages = []
    def clip(self,value,minv,maxv):
        return max(min(value,maxv),minv)
    def move(self,devices):
        self.pos_x = self.clip(self.pos_x + self.dis_move.rvs(),0,self.map[0])
        self.pos_y = self.clip(self.pos_y + self.dis_move.rvs(),0,self.map[1])
        

    def draw(self,ax):
        ax.plot(self.pos_x,self.pos_y,self.color) 
    
    def get_neighbs(self,mat,dev):
        return np.where(mat[dev])[0]
    
    def recive(self):
        for m in self.memory:
            if m.reciver == self.id :
                m.successd = "sent"
                self.messages.append(m)

    


    def send_to_neighbs(self,mat,devices,message,device):
        neighbs = self.get_neighbs(mat,device.id)
        if len(neighbs) > 0 :
            device.battery -=0.01
            device.mp2 = True
            for negh in neighbs:
                devices[negh].memory.append(message)
                
    
    def send(self,devices,mat,message):
        self.send_to_neighbs(mat,devices,message,self)
        return message
    
    
    
    def send2(self,devices,mat,device,message):

        
        neighbs = device.get_neighbs(mat,device.id)

        if not device.mp2 :
                device.send_to_neighbs(mat,devices,message,device)
                for n in neighbs:
                    self.send2(devices,mat,devices[n],message)




        
    
    def protocol1(self,devices,mat,messages,t):
        if self.dis_send.rvs():
            ids = [device.id for device in devices if device.id != self.id]
            id = ids[self.dis_reciver.rvs()]
            message = Message(self.id,id,self.message_type[self.dis_type.rvs()],t)
            messages.append(self.send(devices,mat,message))

        
    def protocol2(self,devices,mat,messages,t):
        if self.dis_send.rvs():
            ids = [device.id for device in devices if device.id != self.id]
            id = ids[self.dis_reciver.rvs()]
            message = Message(self.id,id,self.message_type[self.dis_type.rvs()],t)
            messages.append(message)

            self.send2(devices,mat,self,message)
            
            for device in devices:
                device.mp2 = False
                    
            
    def send3(self,sender,reciver,message):
        reciver.memory.append(message)
        sender.battery -=0.01
            
        


             
    def protocol3(self,devices,mat,messages,t):
        if self.dis_send.rvs():    
            ids = [device.id for device in devices if device.id != self.id]
            id = ids[self.dis_reciver.rvs()]   
            message = Message(self.id,id,self.message_type[self.dis_type.rvs()],t)
            messages.append(message)
            path = bfs(mat,self.id,id)


            if path is not None :
               
                for idx,_ in enumerate(path[:-1]):
                    sender = devices[path[idx]]
                    reciver = devices[path[idx+1]]
                    self.send3(sender,reciver,message)
            else : 
                message.successd = "failed"
                         


    def get_cords(self):
        return self.pos_x,self.pos_y




def sim(protocol=1,N_steps = 3600):
    messages = []
    devices = [Device(i,(100,100),1) for i in range(0,N_devices)]
    # fig, ax = plt.subplots()    

    mat=get_adj(devices)
    for t in range(N_steps):
        # ax.clear()
        mat = get_adj(devices)
        
      
        for i,device in enumerate(devices):
            device.move(devices)

            if protocol == 1:
                device.protocol1(devices,mat,messages,t)
            elif protocol == 2:
                device.protocol2(devices,mat,messages,t)
            elif protocol == 3:
                device.protocol3(devices,mat,messages,t)
            
        #     device.draw(ax)
            
            
        
    
        
        
        # ax.text(1,1,f"{t}/{N_steps}",fontsize = 18, color = "blue")
        # ax.axis([0,100,0,100])
        # plt.pause(0.01)


    for i,device in enumerate(devices):
        device.recive()
        
    data = {
        "message":[m.message for m in messages],
        "sender":[m.sender for m in messages],
        "reciver":[m.reciver for m in messages],
        "successd":[m.successd for m in messages],
        "time":[m.time for m in messages]
    }
    df = pd.DataFrame(data)

    batt = []
    for device in devices:
        batt.append(device.battery)

    return df,batt

#Simulation of Protocol 1

df1,batt = sim(1)

print("1- Proctocl1 :")
print(f'\tNumber of transmitions :{df1.shape[0]}')
if len(df1["successd"].unique())>1:
    print(f'\tPercentage of fails :{df1["successd"].value_counts()["failed"]/df1.shape[0]*100:.2f}%')
else : 
    print("\tNo fails")
print(f'\tBattery mean :{np.mean(batt):.2f}%')
print()


1- Proctocl1 :
	Number of transmitions :7208
	Percentage of fails :64.03%
	Battery mean :96.40%



In [6]:
#Code is on device.protocol2

df2,batt2 = sim(2)
print("2- Proctocl2 :")
print(f'\tNumber of transmitions :{df2.shape[0]:.2f}')

if len(df2["successd"].unique())>1:
    print(f'\tPercentage of fails :{df2["successd"].value_counts()["failed"]/df2.shape[0]*100:.2f}%')
else : 
    print("\tNo fails")

print(f'\tBattery mean :{np.mean(batt2):.2f}%')
print()

2- Proctocl2 :
	Number of transmitions :7254.00
	No fails
	Battery mean :27.46%



In [7]:
#Code is on device.protocol3
df3,batt3 = sim(3)
print("3- Proctocl3 :")

print(f'\tNumber of transmitions :{df3.shape[0]}')

if len(df3["successd"].unique())>1:
    print(f'\tPercentage of fails :{df3["successd"].value_counts()["failed"]/df3.shape[0]*100:.2f}%')
else : 
    print("\tNo fails")
print(f'\tBattery mean :{np.mean(batt3):.2f}%')
print()

3- Proctocl3 :
	Number of transmitions :7133
	Percentage of fails :0.49%
	Battery mean :93.39%



since every device have adjacency matrix we used BFS(Breadth-first search) to get path from sender to reciver and every device send the transmition will loose 0.01 in battery 
so is better than protocol2 in battery 
and since we get directly the path bfs is  better than protocol1 in Percentage of fails  


1- Proctocl1 :
	Number of transmitions :7286
	Percentage of fails :54.54%
	Battery mean :96.37%

2- Proctocl2 :
	Number of transmitions :7123.00
	Percentage of fails :0.60%
	Battery mean :28.10%

3- Proctocl3 :
	Number of transmitions :7291
	No fails
	Battery mean :93.29%

In [8]:
def sim10(protocol=1,battery = 10):
    messages = []
    devices = [Device(i,(100,100),1) for i in range(0,N_devices)]
    mat=get_adj(devices)
    t=0
    while True:
        # ax.clear()
        mat = get_adj(devices)
        
        batt = []
        for device in devices:
            batt.append(device.battery)
        
        if np.mean(batt) <= 10 :
            break
            
      
        for i,device in enumerate(devices):
            device.move(devices)

            if protocol == 1:
                device.protocol1(devices,mat,messages,t)
            elif protocol == 2:
                device.protocol2(devices,mat,messages,t)
            elif protocol == 3:
                device.protocol3(devices,mat,messages,t)
            
            # device.draw(ax)
            
        t+=1
            
        
    
        
        
        # ax.text(1,1,f"{len(devices)}",fontsize = 18, color = "blue")
        # ax.axis([0,100,0,100])
        # plt.pause(0.01)


    for i,device in enumerate(devices):
        device.recive()
    data = {
        "message":[m.message for m in messages],
        "sender":[m.sender for m in messages],
        "reciver":[m.reciver for m in messages],
        "successd":[m.successd for m in messages],
        "time":[m.time for m in messages]
    }
    df = pd.DataFrame(data)


    return df,batt,t

#sim10 will get as argument wich protocol and it will calculate the time until it reaches 10% (Keep in mind it needs the code above)
# we made matplotlib and draw as comments to excute the code faster

df2,bat2,t2 = sim10(2)


print(f"it took :{t2/3600:.2f}h to get {np.mean(bat2):.2f} ")


it took :1.29h to get 9.99 


In [9]:
df3,bat3,t3 = sim10(3)

In [10]:
df3,bat3,t3 = sim10(3) 
print(f"it took :{t3/3600:.2f}h to get {np.mean(bat3):.2f}") 

it took :13.41h to get 10.00
